In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv("../data/AIFeynman/FeynmanEquations.csv")

In [3]:
# Filter all equations with up to 3 independent variables
# df = df[df["# variables"] <= 3]
# df = df.iloc[: len(df)]
# len(df)

In [4]:
# Drop unwanted columns
# df.drop(
#     columns=[
#         "v4_name",
#         "v4_low",
#         "v4_high",
#         "v5_name",
#         "v5_low",
#         "v5_high",
#         "v6_name",
#         "v6_low",
#         "v6_high",
#         "v7_name",
#         "v7_low",
#         "v7_high",
#         "v8_name",
#         "v8_low",
#         "v8_high",
#         "v9_name",
#         "v9_low",
#         "v9_high",
#         "v10_name",
#         "v10_low",
#         "v10_high",
#     ],
#     inplace=True,
# )

In [6]:
df.reset_index(inplace=True, drop=True)

In [7]:
df

,Filename,Number,Output,Formula,# variables,v1_name,v1_low,v1_high,v2_name,v2_low,v2_high,v3_name,v3_low,v3_high
0,I.6.2a,1.0,f,exp(-theta**2/2)/sqrt(2*pi),1.0,theta,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,I.6.2,2.0,f,exp(-(theta/sigma)**2/2)/(sqrt(2*pi)*sigma),2.0,sigma,1.0,3.0,theta,1.0,3.0,NaN,NaN,NaN
2,I.6.2b,3.0,f,exp(-((theta-theta1)/sigma)**2/2)/(sqrt(2*pi)*...,3.0,sigma,1.0,3.0,theta,1.0,3.0,theta1,1.0,3.0
3,I.10.7,6.0,m,m_0/sqrt(1-v**2/c**2),3.0,m_0,1.0,5.0,v,1.0,2.0,c,3.0,10.0
4,I.12.1,8.0,F,mu*Nn,2.0,mu,1.0,5.0,Nn,1.0,5.0,NaN,NaN,NaN
5,I.12.4,11.0,Ef,q1*r/(4*pi*epsilon*r**3),3.0,q1,1.0,5.0,epsilon,1.0,5.0,r,1.0,5.0
6,I.12.5,12.0,F,q2*Ef,2.0,q2,1.0,5.0,Ef,1.0,5.0,NaN,NaN,NaN
7,I.14.3,15.0,U,m*g*z,3.0,m,1.0,5.0,g,1.0,5.0,z,1.0,5.0
8,I.14.4,16.0,U,1/2*k_spring*x**2,2.0,k_spring,1.0,5.0,x,1.0,5.0,NaN,NaN,NaN
9,I.15.1,19.0,p,m_0*v/sqrt(1-v**2/c**2),3.0,m_0,1.0,5.0,v,1.0,2.0,c,3.0,10.0


In [5]:
var_map = {
    "v1_name": "x_1", 
    "v2_name": "x_2", 
    "v3_name": "x_3",
    "v4_name": "x_4",
    "v5_name": "x_5",
    "v6_name": "x_6",
    "v7_name": "x_7",
    "v8_name": "x_8",
    "v9_name": "x_9",
    "v10_name": "x_10",
}

converted_data = []

for i, row in df.iterrows():

    formula = row["Formula"]
    support = {}

    vars = [f'v{i}_' for i in range(1, 11)]

    for var in vars:

        orig_var_name = row[f"{var}name"]

        if type(orig_var_name) is str:
            formula = re.sub(orig_var_name, var_map[f"{var}name"], formula)
            support[var_map[f"{var}name"]] = {
                "max": row[f"{var}high"],
                "min": row[f"{var}low"],
            }

    new_row = {"eq": formula, "support": str(support), "num_points": 500}

    converted_data.append(new_row)

In [6]:
converted_df = pd.DataFrame(data=converted_data)
converted_df.to_csv("../data/AIFeynman/ai_feynman_complete.csv")